In [48]:
import json
import gzip
from collections import defaultdict
from collections import Counter
import glob
import os
import itertools
import sys

In [36]:
def clinvar_jsons():
    with gzip.open("/media/storage/Job_Working_Directory/Python/eva_cttv_pipeline/resources/june16/clinvar_pathlikepath.json.gz", "rt") as f:
        for line in f:
            line = line.rstrip()
            yield json.loads(line)

In [5]:
def is_path_or_likely_path(clinvar_json):
    clin_sigs = set()
    for clinvar_assertion in clinvar_json["clinvarSet"]["clinVarAssertion"]:
        if "description" in clinvar_assertion["clinicalSignificance"]:
            for description in clinvar_assertion["clinicalSignificance"]["description"]:
                clin_sigs.add(description)
        else:
            continue
    if len(clin_sigs.intersection({"Pathogenic", "Likely pathogenic"})) == 0:
        return False
    return True

In [63]:
not_provided_count = 0

In [65]:
def get_trait_names(trait_set):
    global not_provided_count
    trait_list = []
    for trait in trait_set['trait']:
        trait_list.append([])
        for name in trait['name']:
            # First trait name in the list will always be the "Preferred" one
            if name['elementValue']['value'].lower() == "not provided":
                not_provided_count += 1
                continue
            if name['elementValue']['type'] == 'Preferred':
                trait_list[-1] = [name['elementValue']['value']] + trait_list[-1]
            elif name['elementValue']['type'] in ["EFO URL", "EFO id", "EFO name"]:
                continue  # if the trait name not originally from clinvar
            else:
                trait_list[-1].append(name['elementValue']['value'])
    
    trait_names_to_return = []
    for trait in trait_list:
        if len(trait) == 0:
            continue
        trait_names_to_return.append(trait[0].lower())

    return trait_names_to_return

In [29]:
def get_all_trait_names(trait_set):
    trait_list = []
    for trait in trait_set['trait']:
        trait_list.append([])
        for name in trait['name']:
            # First trait name in the list will always be the "Preferred" one
            if name['elementValue']['value'].lower() == "not provided":
                continue
            if name['elementValue']['type'] == 'Preferred':
                trait_list[-1] = [name['elementValue']['value']] + trait_list[-1]
            elif name['elementValue']['type'] in ["EFO URL", "EFO id", "EFO name"]:
                continue  # if the trait name not originally from clinvar
            else:
                trait_list[-1].append(name['elementValue']['value'])
                
    trait_names_to_return = []
    for trait in trait_list:
        if len(trait) == 0:
            continue
        for name in trait:
            trait_names_to_return.append(name.lower())
        
    return trait_names_to_return

First, the number of pathogenic or likely pathogenic records in total... 45715

In [12]:
count = 0
for record in clinvar_jsons():
    if is_path_or_likely_path(record):
        count += 1
print(count)

45715


Create file with just pathogenic and likely pathogenic records to speed up future processing

In [6]:
with gzip.open("/media/storage/Job_Working_Directory/Python/eva_cttv_pipeline/resources/june16/clinvar_pathlikepath.json.gz", "wt") as f:
    for record in clinvar_jsons():
        if is_path_or_likely_path(record):
            f.write(json.dumps(record) + "\n")

Total number of traits (considering records can have multiple traits)... 46316

In [22]:
count = 0
for record in clinvar_jsons():
    for trait in record["clinvarSet"]["referenceClinVarAssertion"]["traitSet"]["trait"]:
        count += 1
print(count)

46316


Find total number of preferred trait names

In [20]:
trait_name_counter = defaultdict(int)
for record in clinvar_jsons():
    record_trait_names = get_trait_names(record["clinvarSet"]["referenceClinVarAssertion"]["traitSet"])
    for trait_name in record_trait_names:
        trait_name_counter[trait_name.lower()] += 1

In [21]:
print(len(trait_name_counter))

5201


Total number of trait names (not just preferred)

In [30]:
trait_names = set()
for record in clinvar_jsons():
    record_trait_names = get_all_trait_names(record["clinvarSet"]["referenceClinVarAssertion"]["traitSet"])
    trait_names.update(record_trait_names)

In [31]:
print(len(trait_names))

10231


Looking at allele origin

In [ ]:
Find if any records have

In [40]:
allele_origins = []
for record in clinvar_jsons():
    record_allele_origins = set()
    for clinvar_assetion_document in record["clinvarSet"]['clinVarAssertion']:
        for observed_in_document in clinvar_assetion_document['observedIn']:
            record_allele_origins.add(observed_in_document['sample']['origin'])
    record_allele_origins = sorted(list(record_allele_origins))
    allele_origins.append(','.join(record_allele_origins))

In [41]:
allele_origins_counter = Counter(allele_origins)

In [42]:
print(allele_origins_counter)

Counter({'germline': 39139, 'unknown': 1699, 'germline,not provided': 1307, 'not provided': 857, 'somatic': 666, 'de novo': 647, 'germline,unknown': 550, 'de novo,germline': 125, 'inherited': 123, 'maternal': 112, 'not provided,unknown': 72, 'germline,inherited': 68, 'paternal': 52, 'germline,not provided,unknown': 39, 'germline,somatic': 37, 'de novo,unknown': 32, 'de novo,germline,unknown': 26, 'germline,maternal': 23, 'germline,paternal': 18, 'biparental': 15, 'maternal,unknown': 9, 'germline,somatic,unknown': 8, 'germline,not applicable': 8, 'not provided,somatic': 7, 'tested-inconclusive': 6, 'de novo,germline,maternal,unknown': 5, 'germline,inherited,not provided': 4, 'paternal,unknown': 4, 'inherited,not provided': 4, 'uniparental': 4, 'inherited,paternal': 3, 'de novo,germline,inherited': 3, 'germline,maternal,not provided': 3, 'de novo,not applicable': 3, 'germline,maternal,unknown': 3, 'de novo,somatic': 2, 'germline,tested-inconclusive': 2, 'maternal,paternal': 2, 'inherited

In [48]:
for ao_combo, count in allele_origins_counter.most_common():
    print('\t'.join([ao_combo, str(count)]))

germline	39139
unknown	1699
germline,not provided	1307
not provided	857
somatic	666
de novo	647
germline,unknown	550
de novo,germline	125
inherited	123
maternal	112
not provided,unknown	72
germline,inherited	68
paternal	52
germline,not provided,unknown	39
germline,somatic	37
de novo,unknown	32
de novo,germline,unknown	26
germline,maternal	23
germline,paternal	18
biparental	15
maternal,unknown	9
germline,somatic,unknown	8
germline,not applicable	8
not provided,somatic	7
tested-inconclusive	6
de novo,germline,maternal,unknown	5
germline,inherited,not provided	4
paternal,unknown	4
inherited,not provided	4
uniparental	4
inherited,paternal	3
de novo,germline,inherited	3
germline,maternal,not provided	3
de novo,not applicable	3
germline,maternal,unknown	3
de novo,somatic	2
germline,tested-inconclusive	2
maternal,paternal	2
inherited,unknown	2
de novo,paternal,unknown	2
de novo,germline,maternal	2
de novo,germline,not provided	2
inherited,maternal	2
de novo,germline,maternal,not provided,unkn

data from clinvar's variant_summary files. below analysis might be best left until json file available....

In [2]:
var_sum_dir = "/home/tom/tmp/clinvar/"

In [20]:
rcv_dict_set = defaultdict(set)
rcv_dict_list = defaultdict(list)

In [6]:
def use_record(record):
    if record[12].lower() != "grch38":
        return False
    if "pathogenic" not in record[5].lower():
        return False
    return True

In [29]:
for filename in os.listdir(var_sum_dir):
    if not filename.startswith("variant_summary_"):
        continue
    with gzip.open(var_sum_dir + filename, "rt") as file:
        next(file)
        for line in file:
            record = line.rstrip().split("\t")
            if not use_record(record):
                continue
            rcvs = record[8].split(";")
            for rcv in rcvs:
                rcv_dict_set[filename].add(rcv)
                rcv_dict_list[filename].append(rcv)

In [32]:
for file, rcv_set in rcv_dict_list.items():
    print(file, len(rcv_set))

variant_summary_2016-04.txt.gz 111025
variant_summary_2016-07.txt.gz 116291
variant_summary_2016-06.txt.gz 113806
variant_summary_2016-08.txt.gz 118750
variant_summary_2016-01.txt.gz 107385
variant_summary_2016-02.txt.gz 108618
variant_summary_2016-09.txt.gz 119421
variant_summary_2016-05.txt.gz 111508
variant_summary_2016-03.txt.gz 109279


In [33]:
for month in range(2,10):
    last_month = month - 1
    last_month_file = "variant_summary_2016-0" + str(last_month) + ".txt.gz"
    this_month_file = "variant_summary_2016-0" + str(month) + ".txt.gz"
    num_removed = len(rcv_dict_set[last_month_file] - rcv_dict_set[this_month_file])    
    num_added = len(rcv_dict_set[this_month_file] - rcv_dict_set[last_month_file])
    print("Month {} to {}:".format(last_month, month))
    print("Removed: {}".format(num_removed))
    print("Added: {}".format(num_added))
    print()

Month 1 to 2:
Removed: 164
Added: 990

Month 2 to 3:
Removed: 191
Added: 626

Month 3 to 4:
Removed: 95
Added: 1187

Month 4 to 5:
Removed: 168
Added: 501

Month 5 to 6:
Removed: 1888
Added: 3872

Month 6 to 7:
Removed: 677
Added: 2478

Month 7 to 8:
Removed: 777
Added: 2732

Month 8 to 9:
Removed: 73
Added: 447



In [24]:
print(len(rcv_dict_list["variant_summary_2016-09.txt.gz"]))
print(len(rcv_dict_set["variant_summary_2016-09.txt.gz"]))

52550
52295


In [27]:
rcv_list_counter_07 = Counter(rcv_dict_list["variant_summary_2016-07.txt.gz"])

In [28]:
print(rcv_list_counter_07.most_common(2))

[('RCV000149583;RCV000170535;RCV000170536;RCV000170538;RCV000170539;RCV000170540;RCV000170541;RCV000170542;RCV000170543', 9), ('RCV000016286;RCV000030905;RCV000016575;RCV000016573;RCV000016574;RCV000224000;RCV000016576;RCV000016577;RCV000016579;RCV000016580;RCV000016877;RCV000016879', 9)]


In [38]:
def get_rcv(record):
    return record["clinvarSet"]["referenceClinVarAssertion"]["clinVarAccession"]["acc"]

In [54]:
def get_rs_ids(record):
    rs_ids = set()
    measures = record["clinvarSet"]["referenceClinVarAssertion"]["measureSet"]["measure"]
    for measure in measures:
        if "xref" not in measure:
            continue
        xref = measure["xref"]
        for ref in xref:
            if ref["db"].lower() != "dbsnp":
                continue
            if ref["type"].lower() != "rs":
                continue
            rs_ids.add("rs{}".format(ref["id"]))
    if len(rs_ids) == 0:
        rs_ids = {"-1"}
    return list(rs_ids)

In [66]:
# with gzip.open("/media/storage/Job_Working_Directory/Python/eva_cttv_pipeline/resources/oct16/mapping/for_zooma/rcv_rs_traitname_A.tsv.gz", 
#                "wt") as outfile:
for record in clinvar_jsons():
    rcv = get_rcv(record)
    rs_ids = get_rs_ids(record)
    trait_names = get_trait_names(record["clinvarSet"]["referenceClinVarAssertion"]["traitSet"])
#     if len(trait_names) < 1:
#         print(record)
#         sys.exit(1)
    for rs_id, trait_name in itertools.product(rs_ids, trait_names):
        output_line = "\t".join([rcv, rs_id, trait_name])
#             outfile.write(output_line + "\n")

print(not_provided_count)

4861


In [56]:
rcvs = set()
for record in clinvar_jsons():
    rcv = get_rcv(record)
    rcvs.add(rcv)
    
print(len(rcvs))

45715
